In [ ]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from scipy.stats import ttest_ind

In [2]:
customer_panel_df = pd.read_csv('/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv')

In [3]:
# What are the proportions of the responses?
n = customer_panel_df.shape[0]
prop_default = (customer_panel_df['DefaultLabel'] == 1).sum() / n
print(f' Proportion of customers who default: {prop_default}')
print(f' Proportion of customers who do not default: {1-prop_default}')
# Dataset is extremely imbalanced - only 9.2% of customers in the data default

 Proportion of customers who default: 0.09232388542214871
 Proportion of customers who do not default: 0.9076761145778512


In [3]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Import train and test data
train_data = pd.read_csv('Additional_train.csv')


# For SMOTE:
# smote = SMOTE(random_state=42, k_neighbors=5, sampling_strategy=0.3)
# smote = BorderlineSMOTE(kind='borderline-1', random_state=42, k_neighbors=5)
# train_data = train_data.fillna(-1)

features = ['PaymentRatio', 'TotalBalance', 'Limit', 'NumCredit',
            'NumSavings', 'Age', 'CreditScore', 'HomeCity', 'AnnualSalary', 
            'balance_to_limit', 'total_accounts', 'missed_payment_flag', 
            'payment_ratio_change', 'credit_to_savings_ratio', 'balance_to_salary','limit_to_salary', 
            'payment_to_salary_ratio', 'is_high_inquiry_user', "PaymentRatio_std", 'Utilisation_mean',
            'Utilisation_std', 'Utilisation_max','PaymentRatio_mean',
            'PaymentRatio_std', "HardInquiries_mean", "balance_to_limit_mean", 'HardInquiries_sum',
           'balance_to_limit_max' ]
X_train_df = train_data[features]
y_train_df = train_data['DefaultLabel']
X, y = X_train_df.to_numpy(), y_train_df.to_numpy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify = y, random_state=42)

X_train = np.nan_to_num(X_train, nan=-1)
X_val = np.nan_to_num(X_val, nan=-1)
smote = BorderlineSMOTE(kind='borderline-1', random_state=42, k_neighbors=5)

test_data = pd.read_csv('Additional_test.csv')
X_test_df = test_data[features]
X_test = X_test_df.to_numpy()

# X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [40]:
# for feature in features:
#     group1 = train_data[train_data['DefaultLabel']==1].dropna()
#     group2 = train_data[train_data['DefaultLabel']==0].dropna()
#     ttest = ttest_ind(group1[feature], group2[feature])
#     print(feature)
#     print(f'T-Stat: {ttest.statistic}')
#     print(f'p-val: {ttest.pvalue}')
#     print('--------------')

In [ ]:
neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
scale_pos_weight = neg_count / pos_count
param_grid = {
    # Tree complexity
    'max_depth': [4, 5],
    'min_child_weight': [3, 6, 9],

    'gamma': [1.0, 2.0, 4.0],

    # Sampling
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    
    'eta': [0.03, 0.05],

    'reg_lambda': [2.0, 5.0, 10.0],
    'reg_alpha': [0.0, 0.5, 1.0],
    
}

xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='aucpr',
    random_state=42,
    tree_method='hist',
    n_estimators = 20000,
    early_stopping_rounds = 200,
    scale_pos_weight = scale_pos_weight
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='f1_macro',
    cv=cv,
    verbose=2,
    n_jobs=1,
)

grid.fit(X_train, y_train, eval_set=[(X_val, y_val)])

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[0]	validation_0-aucpr:0.50831
[1]	validation_0-aucpr:0.52648
[2]	validation_0-aucpr:0.53720
[3]	validation_0-aucpr:0.58731
[4]	validation_0-aucpr:0.65064
[5]	validation_0-aucpr:0.69624
[6]	validation_0-aucpr:0.68095
[7]	validation_0-aucpr:0.68673
[8]	validation_0-aucpr:0.68898
[9]	validation_0-aucpr:0.68579
[10]	validation_0-aucpr:0.70942
[11]	validation_0-aucpr:0.70136
[12]	validation_0-aucpr:0.69660
[13]	validation_0-aucpr:0.69823
[14]	validation_0-aucpr:0.69529
[15]	validation_0-aucpr:0.70101
[16]	validation_0-aucpr:0.70711
[17]	validation_0-aucpr:0.70748
[18]	validation_0-aucpr:0.69170
[19]	validation_0-aucpr:0.70214
[20]	validation_0-aucpr:0.70244
[21]	validation_0-aucpr:0.70889
[22]	validation_0-aucpr:0.70680
[23]	validation_0-aucpr:0.71232
[24]	validation_0-aucpr:0.71358
[25]	validation_0-aucpr:0.71701
[26]	validation_0-aucpr:0.72257
[27]	validation_0-aucpr:0.72216
[28]	validation_0-aucpr:0.71918
[29]	validation_0

In [ ]:
print("Best Params:", grid.best_params_)

In [54]:
grid.best_score_

np.float64(0.8201908060839631)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Evaluate performance on train and validation sets using Macro F1-Score
y_hat_train = xgb.predict(X_train)
y_hat_val = xgb.predict(X_val)

F1_train = f1_score(y_train, y_hat_train, average='macro')
precision_train = precision_score(y_train, y_hat_train)
recall_train = recall_score(y_train, y_hat_train)

F1_val = f1_score(y_val, y_hat_val, average='macro')
precision_val = precision_score(y_val, y_hat_val)
recall_val = recall_score(y_val, y_hat_val)

print(f'F1-Score on Training Set: {F1_train}')
print(f'Precision on Training Set: {precision_train}')
print(f'Recall on Training Set: {recall_train}')
print('--------------------------------')
print(f'F1-Score on Validation Set: {F1_val}')
print(f'Precision on Validation Set: {precision_val}')
print(f'Recall on Validation Set: {recall_val}')

In [53]:
from agentds import BenchmarkClient

# Generate submission file
client = BenchmarkClient(
    api_key="adsb_hdm1DRk1iW2I1VA84Oc9jz6z_1756090832",
    team_name="agi"
)

# Make predictions
best_model = grid.best_estimator_
predictions = best_model.predict(X_test)
# predictions = xgb.predict(X_test)

# Create submission file (format: CustomerID,Week,DefaultLabel)
submission_df = pd.DataFrame({
    'CustomerID': test_data['CustomerID'],
    'Week': test_data['Week'],
    'DefaultLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge2_predictions2.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Default rate: {predictions.mean():.3f} ({predictions.sum()} defaults out of {len(predictions)})")

✅ Predictions saved: 13290 predictions
   Preview:   CustomerID  Week  DefaultLabel
0    C000001     1             0
1    C000001     2             0
2    C000001     3             0
   Default rate: 0.035 (459 defaults out of 13290)


In [55]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_predictions2.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. You've completed all Retail Banking challenges!")

🚀 Submitting predictions...
✅ Prediction submitted successfully!
📊 Score: 0.5528 (Macro-F1)
✅ Validation passed
✅ Submission successful!
   📊 Score: 0.5528
   📏 Metric: Macro-F1
   ✔️  Validation: Passed

🎯 Next steps:
   1. Try incorporating relevant information outside this table!
   2. You've completed all Retail Banking challenges!


In [ ]:
# Which of the features are highly correlated with each other?

In [95]:
from scipy.stats import ttest_ind
# Which of the features are highly associated with the response?
# For quantitative predictors use T-test to see if mean values differ across the response
for feature in features:
    group1 = train_data[train_data['DefaultLabel']==1].dropna()
    group2 = train_data[train_data['DefaultLabel']==0].dropna()
    ttest = ttest_ind(group1[feature], group2[feature])
    print(feature)
    print(f'T-Stat: {ttest.statistic}')
    print(f'p-val: {ttest.pvalue}')
    print('--------------')

# Can drop Utilisation, HardInquiries, NumChecking, Tenure
    

Utilisation
T-Stat: 0.9781635715541426
p-val: 0.3280357898449866
--------------
PaymentRatio
T-Stat: -2.67661840527117
p-val: 0.007458648207991715
--------------
HardInquiries
T-Stat: 0.22258954176421403
p-val: 0.8238631147044173
--------------
TotalBalance
T-Stat: -8.959888135601098
p-val: 4.3556564698963365e-19
--------------
Limit
T-Stat: -7.755840430168055
p-val: 1.0361813115089424e-14
--------------
NumCredit
T-Stat: -7.016270057249607
p-val: 2.549192942925729e-12
--------------
NumChecking
T-Stat: 1.032929473718969
p-val: 0.3016816217607109
--------------
NumSavings
T-Stat: -4.847742974358312
p-val: 1.2825486798681206e-06
--------------
Age
T-Stat: -8.659095128779944
p-val: 6.14264114140903e-18
--------------
Tenure
T-Stat: -0.22535794029225914
p-val: 0.8217090726602149
--------------
CreditScore
T-Stat: 2.5545274303419716
p-val: 0.010659497396642419
--------------
HomeCity
T-Stat: 47.91219093870581
p-val: 0.0
--------------
AnnualSalary
T-Stat: -9.67821500049923
p-val: 5.5482184